## Ingestion del archivo JSON multi-linhas "movie_cast.json"

#### `Paso 0 - Actualización de variables y funciones y validaciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
#Chama a função para validar se o arquivo a ser processado existe
valida_arquivo(bronze_folder_path, v_file_date, "movie_cast.json")

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/bronze

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
#Definindo o schema na variavel name_schema
movies_casts_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("genderId", IntegerType(), True),
    StructField("castOrder", IntegerType(), True)
])

In [0]:
movies_casts_df = spark.read \
               .schema(movies_casts_schema) \
               .option("multiLine", True) \
               .json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json")

In [0]:
#display(movies_casts_df)

#### Paso 2 - Renombrar el nombre de las columnas y añadir "ingestion_date" y "enviroment"
1. "movieid" renombrar a "person_id"
2. "personId renombrar a "person_id"
3. "characterName" renombrar a "character_name
4. Agregar las columnas "ingestion_date" y "Enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_casts_with_columns_df = add_ingestion_date(movies_casts_df) \
                    .withColumnsRenamed({"movieId": "movie_id", 
                                         "personId": "person_id", 
                                         "characterName": "character_name"}) \
                    .withColumn("enviroment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
#display(movies_casts_with_columns_df)

#### Paso 3 - Eliminar las columnas no deseadas del DataFrame

In [0]:
from pyspark.sql.functions import col
movies_casts_final_df = movies_casts_with_columns_df.drop(col("genderId"), col("castOrder"))

In [0]:
#display(movies_casts_final_df)

#### Paso 4 - Escribir datos en el datalake en formato parquet

In [0]:
#Elimina partição se ela já existir
#drop_partition_if_exists(movies_casts_final_df, "movie_silver", "movies_casts", "file_date")

In [0]:
#movies_casts_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_casts/")
#movies_casts_final_df.write.mode("append") \
#                        .partitionBy("file_date") \
#                        .format("parquet") \
#                        .saveAsTable("movie_silver.movies_casts")

merge_delta_lake(movies_casts_final_df, "movie_silver", "movies_casts", silver_folder_path, "tgt.movie_id = src.movie_id AND tgt.person_id = src.person_id", "file_date")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/movies_casts

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_casts"))
#spark.sql("select * from movie_silver.movies_casts").display()
display(spark.sql("SELECT file_date, \
                          COUNT(1) \
                   FROM movie_silver.movies_casts \
                   GROUP BY file_date"
                   )
        )

In [0]:
dbutils.notebook.exit("Success")